In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models-transformers-net-item-prediction/nvidia_logo.png" style="width: 90px; float: right;">

# Transformer-based architecture for next-item prediction task

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container.

## Overview

In this use case we will train a Transformer-based architecture for next-item prediction task.

**Note, the data for this notebook will be automatically downloaded to the folder specified in the cells below.**

We will use the [booking.com dataset](https://github.com/bookingcom/ml-dataset-mdt) to train a session-based model. The dataset contains 1,166,835 of anonymized hotel reservations in the train set and 378,667 in the test set. Each reservation is a part of a customer's trip (identified by `utrip_id`) which includes consecutive reservations.

We will reshape the data to organize it into 'sessions'. Each session will be a full customer itinerary in chronological order. The goal will be to predict the city_id of the final reservation of each trip.


### Learning objectives

- Training a Transformer-based architecture for next-item prediction task

## Downloading and preparing the dataset

We will download the dataset using a functionality provided by merlin models. The dataset can be found on GitHub [here](https://github.com/bookingcom/ml-dataset-mdt).

**Read more about libraries used in the import statements below**

- [get_lib](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/core/dispatch.py)
- [get_booking](https://github.com/NVIDIA-Merlin/models/tree/main/merlin/datasets/ecommerce)
- [nvtabular](https://github.com/NVIDIA-Merlin/NVTabular/tree/main/nvtabular)
- [nvtabular ops](https://github.com/NVIDIA-Merlin/NVTabular/tree/main/nvtabular/ops)
- [schema tags](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/tags.py)
- [merlin models tensorflow](https://github.com/NVIDIA-Merlin/models/tree/main/merlin/models/tf)
- [get_booking](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/datasets/ecommerce/booking/dataset.py)

In [2]:
# Resetting the TF memory allocation to not be 50% by default. 
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

from merlin.core.dispatch import get_lib
from merlin.datasets.ecommerce import get_booking

import numpy as np

from nvtabular import *
from nvtabular import ops

from merlin.schema.tags import Tags
import merlin.models.tf as mm

get_booking('/workspace/data')
train = get_lib().read_csv('/workspace/data/train_set.csv', parse_dates=['checkin', 'checkout'])

print('Training data import complete')

2023-04-19 01:35:02.207465: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2023-04-19 01:35:03.422817: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-19 01:35:03.423265: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-19 01:35:03.423447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-04-19 01:35:03.695996: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 01:35:03.696824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-19 01:35:03.697038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-19 01:35:03.697196: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

Training data import complete


/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.SESSION_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.SESSION: 'session'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Each reservation has a unique `utrip_id`. During each trip a customer vists several destinations.

In [3]:
# When displaying cudf dataframes use print() or display(), otherwise Jupyter creates hidden copies.
print(train.head())

   user_id    checkin   checkout  city_id device_class  affiliate_id  \
0  1000027 2016-08-13 2016-08-14     8183      desktop          7168   
1  1000027 2016-08-14 2016-08-16    15626      desktop          7168   
2  1000027 2016-08-16 2016-08-18    60902      desktop          7168   
3  1000027 2016-08-18 2016-08-21    30628      desktop           253   
4  1000033 2016-04-09 2016-04-11    38677       mobile           359   

  booker_country hotel_country   utrip_id  
0        Elbonia        Gondal  1000027_1  
1        Elbonia        Gondal  1000027_1  
2        Elbonia        Gondal  1000027_1  
3        Elbonia        Gondal  1000027_1  
4         Gondal  Cobra Island  1000033_1  


We will train on sequences of `city_id` and `booker_country` and based on this information, our model will attempt to predict the next `city_id` (the next hop in the journey).

We will train a transformer model that can work with sequences of variable length within a batch. This functionality is provided to us out of the box and doesn't require any changes to the architecture. Thanks to it we do not have to pad or trim our sequences to any particular length -- our model can make effective use of all of the data!

*With one exception.* For a masked language model that we will be training, we need to discard sequences that are shorter than two hops. This makes sense as there is nothing our model could learn if it was only presented with an itinerary with a single destination on it!

Let us begin by splitting the data into a train and validation set based on trip ID.

Let's see how many unique trips there are in the dataset. Also, let us shuffle the trips along the way so that our validation set consists of a random sample of our train set.

In [4]:
# Unique trip ids.
utrip_ids = train.sample(frac=1).utrip_id.unique()
print('Number of unique trips is :', len(utrip_ids))

Number of unique trips is : 217686


Now let's assign data to our train and validation sets. Furthermore, we sort the data by `utrip_id` and `checkin`. This way we ensure our sequences of visited `city_ids` will be in proper order!

Also, let's remove trips where only a single city was visited as they cannot be modeled as a sequence.

In [5]:
train = get_lib().from_pandas(
    train.to_pandas().join(train.to_pandas().groupby('utrip_id').size().rename('num_examples'), on='utrip_id')
)
train = train[train.num_examples > 1]

train.checkin = train.checkin.astype('int')
train.checkout = train.checkout.astype('int')

train_set_utrip_ids = utrip_ids[:int(0.8 * utrip_ids.shape[0])]
validation_set_utrip_ids = utrip_ids[int(0.8 * utrip_ids.shape[0]):]

train_set = train[train.utrip_id.isin(train_set_utrip_ids)].sort_values(['utrip_id', 'checkin'])
validation_set = train[train.utrip_id.isin(validation_set_utrip_ids)].sort_values(['utrip_id', 'checkin'])

##  Preprocessing with NVTabular

We can now begin with data preprocessing.

We will combine trips into "sessions", discard trips that are too short and calculate total trip length.

We will use NVTabular for this work. It offers optimized tabular data preprocessing operators that run on the GPU. If you would like to learn more about the NVTabular library, please take a look [here](https://github.com/NVIDIA-Merlin/NVTabular).

Read more about the [Merlin's Dataset API](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/io/dataset.py)  
Read more about how [parquet files are read in and processed by Merlin](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/io/parquet.py)  
Read more about [Tags](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/tags.py)  
- [schema_select_by_tag](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)  

Read more about [NVTabular Workflows](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/workflow/workflow.py)  
- [fit_transform](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/workflow/workflow.py)
- [transform](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/workflow/workflow.py)  

Read more about the [NVTabular Operators]()  
- [Categorify](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/categorify.py)
- [AddTags](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/add_metadata.py)
- [LambdaOp](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/lambdaop.py)
- [Rename](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/rename.py)
- [Filter](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/nvtabular/ops/filter.py)



In [6]:
train_set_dataset = Dataset(train_set)
validation_set_dataset = Dataset(validation_set)

In [7]:
weekday_checkin = (
    ["checkin"]
    >> ops.LambdaOp(lambda col: get_lib().to_datetime(col).dt.weekday)
    >> ops.Rename(name="weekday_checkin")
)

weekday_checkout = (
    ["checkout"]
    >> ops.LambdaOp(lambda col: get_lib().to_datetime(col).dt.weekday)
    >> ops.Rename(name="weekday_checkout")
)

categorical_features = (['city_id', 'booker_country', 'hotel_country'] +
                         weekday_checkin + weekday_checkout
                       ) >> ops.Categorify(start_index=1)  

groupby_features = categorical_features + ['utrip_id', 'checkin'] >> ops.Groupby(
    groupby_cols=['utrip_id'],
    aggs={
        'city_id': ['list', 'count'],
        'booker_country': ['list'],
        'hotel_country': ['list'],
        'weekday_checkin': ['list'],
        'weekday_checkout': ['list']
    },
    sort_cols="checkin"
)

list_features = (
            groupby_features['city_id_list', 'booker_country_list', 'hotel_country_list', 
                 'weekday_checkin_list', 'weekday_checkout_list'
            ] >> ops.AddTags([Tags.SEQUENCE])
)

# Filter out sessions with less than 2 interactions 
MINIMUM_SESSION_LENGTH = 2
features = list_features + (groupby_features['city_id_count'] >>  ops.AddTags([Tags.CONTINUOUS]))
filtered_sessions = features >> ops.Filter(f=lambda df: df["city_id_count"] >= MINIMUM_SESSION_LENGTH) 

In [8]:
wf = Workflow(filtered_sessions)

In [9]:
train_set_processed = wf.fit_transform(train_set_dataset)
validation_set_processed = wf.transform(validation_set_dataset)

Our data consists of a sequence of visited `city_ids`, a sequence of `booker_countries` (represented as integer categories) and a `city_id_count` column (which contains the count of visited cities in a trip).

In [10]:
train_set_processed.compute().head()

,city_id_list,booker_country_list,hotel_country_list,weekday_checkin_list,weekday_checkout_list,city_id_count
0,"[8239, 157, 2279, 2098]","[4, 4, 4, 4]","[4, 4, 4, 4]","[6, 8, 5, 4]","[8, 5, 3, 8]",4
1,"[64, 1161, 88, 619, 64]","[2, 2, 2, 2, 2]","[2, 2, 2, 2, 2]","[6, 2, 5, 4, 6]","[7, 5, 3, 6, 5]",5
2,"[8, 7, 25, 1051, 66, 53, 4]","[3, 3, 3, 3, 3, 3, 3]","[3, 3, 3, 17, 17, 4, 4]","[6, 2, 3, 7, 6, 8, 5]","[7, 4, 2, 6, 8, 5, 4]",7
3,"[1033, 758, 141, 4]","[3, 3, 3, 3]","[20, 20, 20, 4]","[2, 5, 3, 4]","[5, 4, 3, 6]",4
4,"[3604, 263, 663, 251, 360]","[2, 2, 2, 2, 2]","[31, 31, 31, 31, 31]","[2, 4, 7, 6, 2]","[3, 2, 6, 7, 4]",5


We are now ready to train our model.

Here is the schema of the data that our model will use.

In [11]:
seq_schema = train_set_processed.schema.select_by_tag(Tags.SEQUENCE)
seq_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,city_id_list,"(Tags.CATEGORICAL, Tags.SEQUENCE)","DType(name='int64', element_type=<ElementType....",True,True,None,0,0,1,.//categories/unique.city_id.parquet,0,37203,city_id,37204,512,0,None
1,booker_country_list,"(Tags.CATEGORICAL, Tags.SEQUENCE)","DType(name='int64', element_type=<ElementType....",True,True,None,0,0,1,.//categories/unique.booker_country.parquet,0,6,booker_country,7,16,0,None
2,hotel_country_list,"(Tags.CATEGORICAL, Tags.SEQUENCE)","DType(name='int64', element_type=<ElementType....",True,True,None,0,0,1,.//categories/unique.hotel_country.parquet,0,195,hotel_country,196,31,0,None
3,weekday_checkin_list,"(Tags.CATEGORICAL, Tags.SEQUENCE)","DType(name='int64', element_type=<ElementType....",True,True,None,0,0,1,.//categories/unique.weekday_checkin.parquet,0,8,weekday_checkin,9,16,0,None
4,weekday_checkout_list,"(Tags.CATEGORICAL, Tags.SEQUENCE)","DType(name='int64', element_type=<ElementType....",True,True,None,0,0,1,.//categories/unique.weekday_checkout.parquet,0,8,weekday_checkout,9,16,0,None


Align the schema of train and validation datasets with the model's schema

In [12]:
train_set_processed.schema = seq_schema
validation_set_processed.schema = seq_schema

Let's also identify the target column.

In [13]:
target = train_set_processed.schema.select_by_tag(Tags.SEQUENCE).column_names[0]
target

'city_id_list'

## Constructing the model

Let's construct our model.

We can specify various hyperparameters, such as the number of heads and number of layers to use.

For the transformer portion of our model, we will use the `XLNet` architecture.

Later, when we run the `fit` method on our model, we will specify the `masking_probability` of `0.3` and link it to the transformer block defined in out model. Through the combination of these parameters, our model will train on sequences where any given timestep will be masked with a probability of 0.3 and it will be our model's training task to infer the target value for that step!

To summarize, Masked Language Modeling is implemented by:

* `SequenceMaskRandom()` - Used as a pre for model.fit(), it randomly selects items from the sequence to be masked for prediction as targets, by using Keras masking. This block also adds the necessary configuration to the specified `transformer` block so as it
is pre-configured with the necessary layers needed to prepare the inputs to the HuggingFace transformer layer and to post-process its outputs. For example, one pre-processing operation is to replace the input embeddings at masked positions for prediction by a dummy trainable embedding, to avoid leakage of the targets.


**Read more about the apis used to construct models** 
- [blocks](https://github.com/NVIDIA-Merlin/models/tree/main/merlin/models/tf/blocks)
- [MLPBlock](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/blocks/mlp.py)
- [InputBlockV2](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/inputs/base.py)
- [Embeddings](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/inputs/embedding.py)
- [XLNetBlock](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/transformers/block.py)
- [CategoricalOutput](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/outputs/classification.py)
- [.schema.select_by_name](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)
- [.schema.select_by_tag](https://github.com/NVIDIA-Merlin/core/blob/main/merlin/schema/schema.py)
- [model.compile()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/models/base.py)
- [model.fit()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/models/base.py)
- [model.evaluate()](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/models/base.py)
- [mm.SequenceMaskRandom](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/transforms/sequence.py)
- [mm.SequenceMaskLast](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/transforms/sequence.py)

In [14]:
dmodel=48
mlp_block = mm.MLPBlock(
                [128,dmodel],
                activation='relu',
                no_activation_last_layer=True,
            )
transformer_block = mm.XLNetBlock(d_model=dmodel, n_head=4, n_layer=2)
model = mm.Model(
    mm.InputBlockV2(
        seq_schema,
        embeddings=mm.Embeddings(
            train_set_processed.schema.select_by_tag(Tags.CATEGORICAL), sequence_combiner=None
        ),
    ),
    mlp_block,
    transformer_block,
    mm.CategoricalOutput(
        train_set_processed.schema.select_by_name(target),
        default_loss="categorical_crossentropy",
    ),
)

## Model training

In [15]:
model.compile(run_eagerly=False, optimizer='adam', loss="categorical_crossentropy")
model.fit(train_set_processed, batch_size=64, epochs=5, pre=mm.SequenceMaskRandom(schema=seq_schema, target=target, masking_prob=0.3, transformer=transformer_block))

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2023-04-19 01:35:13.112516: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 48), dtype=float32), dense_shape=Tensor("gradient_tape/model/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/xl_net_block/sequential_block_5/replace_masked_embeddings/RaggedWhere/Reshape_3:0", shape=(None,), dtype=int64), values=Tensor("

2023-04-19 01:35:23.514177: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/xl_net_block/sequential_block_5/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_95


2720/2720 [==============================] - 77s 24ms/step - loss: 7.3177 - recall_at_10: 0.2005 - mrr_at_10: 0.0877 - ndcg_at_10: 0.1141 - map_at_10: 0.0877 - precision_at_10: 0.0201 - regularization_loss: 0.0000e+00 - loss_batch: 7.3157
Epoch 2/5
2720/2720 [==============================] - 67s 24ms/step - loss: 6.1038 - recall_at_10: 0.3626 - mrr_at_10: 0.1703 - ndcg_at_10: 0.2157 - map_at_10: 0.1703 - precision_at_10: 0.0363 - regularization_loss: 0.0000e+00 - loss_batch: 6.1022
Epoch 3/5
2720/2720 [==============================] - 63s 23ms/step - loss: 5.5767 - recall_at_10: 0.4321 - mrr_at_10: 0.2067 - ndcg_at_10: 0.2600 - map_at_10: 0.2067 - precision_at_10: 0.0432 - regularization_loss: 0.0000e+00 - loss_batch: 5.5751
Epoch 4/5
2720/2720 [==============================] - 64s 23ms/step - loss: 5.3249 - recall_at_10: 0.4631 - mrr_at_10: 0.2223 - ndcg_at_10: 0.2792 - map_at_10: 0.2223 - precision_at_10: 0.0463 - regularization_loss: 0.0000e+00 - loss_batch: 5.3230
Epoch 5/5
2720

## Model evaluation

We have trained our model.

But in training the metrics come from a masked language modelling task. A portion of steps in the sequence was masked for each example. The metrics were calculated on this task.

In reality, we probably care how well our model does on the next item prediction task (as it mimics the scenario in which the model would be likely to be used).

Let's measure the performance of the model on a task where it attempts to predict the last item in a sequence.

We will mask the last item using `SequenceMaskLast` and run inference.

In [16]:
model.evaluate(
    validation_set_processed,
    batch_size=128,
    pre=mm.SequenceMaskLast(schema=validation_set_processed.schema, target=target, transformer=transformer_block),
    return_dict=True
)

2023-04-19 01:40:55.868086: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/xl_net_block/sequential_block_5/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_71


340/340 [==============================] - 10s 18ms/step - loss: 4.7317 - recall_at_10: 0.5562 - mrr_at_10: 0.3114 - ndcg_at_10: 0.3695 - map_at_10: 0.3114 - precision_at_10: 0.0556 - regularization_loss: 0.0000e+00 - loss_batch: 4.7313


{'loss': 4.731719017028809,
 'recall_at_10': 0.556216835975647,
 'mrr_at_10': 0.3114089369773865,
 'ndcg_at_10': 0.36954960227012634,
 'map_at_10': 0.3114089369773865,
 'precision_at_10': 0.0556216724216938,
 'regularization_loss': 0.0,
 'loss_batch': 4.579279899597168}

## Serving predictions using the Triton Inference Server

Now, we will serve our trained models on [NVIDIA Triton Inference Server (TIS)](https://github.com/triton-inference-server/server). TIS is an open-source inference serving software that helps standardize model deployment and execution and delivers fast and scalable AI in production. To serve recommender models on TIS easily, NVIDIA Merlin team designed and developed [the Merlin Systems library](https://github.com/NVIDIA-Merlin/systems). Merlin Systems provides tools and operators to be able to serve end-to-end recommender systems pipelines on TIS easily

In order to perform inference on the Triton Inference Server, we need to output the inference operators to disk.

The inference operators form an `Ensemble`, which is a pipeline that takes in raw data, processes it using NVTabular, and finally outputs predictions from the model that we trained.

Let's write the `Ensemble` to disk (we will later load it on Triton to perform inference).

In [17]:
from merlin.systems.dag.ops.tensorflow import PredictTensorflow
from merlin.systems.dag.ensemble import Ensemble
from merlin.systems.dag.ops.workflow import TransformWorkflow

inf_ops = wf.input_schema.column_names >> TransformWorkflow(wf) >> PredictTensorflow(model)

ensemble = Ensemble(inf_ops, wf.input_schema)
ensemble.export('/workspace/ensemble');

  (_feature_shapes): Dict(
    (city_id_list): TensorShape([64, None, 1])
    (booker_country_list): TensorShape([64, None, 1])
    (hotel_country_list): TensorShape([64, None, 1])
    (weekday_checkin_list): TensorShape([64, None, 1])
    (weekday_checkout_list): TensorShape([64, None, 1])
  )
  (_feature_dtypes): Dict(
    (city_id_list): tf.int64
    (booker_country_list): tf.int64
    (hotel_country_list): tf.int64
    (weekday_checkin_list): tf.int64
    (weekday_checkout_list): tf.int64
  )
), because it is not built.
  (_feature_shapes): Dict(
    (city_id_list): TensorShape([64, None, 1])
    (booker_country_list): TensorShape([64, None, 1])
    (hotel_country_list): TensorShape([64, None, 1])
    (weekday_checkin_list): TensorShape([64, None, 1])
    (weekday_checkout_list): TensorShape([64, None, 1])
  )
  (_feature_dtypes): Dict(
    (city_id_list): tf.int64
    (booker_country_list): tf.int64
    (hotel_country_list): tf.int64
    (weekday_checkin_list): tf.int64
    (weekd

INFO:tensorflow:Assets written to: /tmp/tmpk2jcmm5x/model.savedmodel/assets


INFO:tensorflow:Assets written to: /tmp/tmpk2jcmm5x/model.savedmodel/assets
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/utils/tf_utils.py:101: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[key] = tf.keras.utils.serialize_keras_object(maybe_value)
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/core/combinators.py:288: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[i] = tf.keras.utils.serialize_keras_object(layer)
/usr/local/lib/python3.8/dist-packages/keras/saving/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_obj

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


/usr/local/lib/python3.8/dist-packages/merlin/systems/dag/node.py:100: UserWarning: Operator 'TransformWorkflow' is producing the output column 'city_id_count', which is not being used by any downstream operator in the ensemble graph.
  warnings.warn(


...

  (_feature_shapes): Dict(
    (city_id_list): TensorShape([64, None, 1])
    (booker_country_list): TensorShape([64, None, 1])
    (hotel_country_list): TensorShape([64, None, 1])
    (weekday_checkin_list): TensorShape([64, None, 1])
    (weekday_checkout_list): TensorShape([64, None, 1])
  )
  (_feature_dtypes): Dict(
    (city_id_list): tf.int64
    (booker_country_list): tf.int64
    (hotel_country_list): tf.int64
    (weekday_checkin_list): tf.int64
    (weekday_checkout_list): tf.int64
  )
), because it is not built.


  (_feature_shapes): Dict(
    (city_id_list): TensorShape([64, None, 1])
    (booker_country_list): TensorShape([64, None, 1])
    (hotel_country_list): TensorShape([64, None, 1])
    (weekday_checkin_list): TensorShape([64, None, 1])
    (weekday_checkout_list): TensorShape([64, None, 1])
  )
  (_feature_dtypes): Dict(
    (city_id_list): tf.int64
    (booker_country_list): tf.int64
    (hotel_country_list): tf.int64
    (weekday_checkin_list): tf.int64
    (weekday_checkout_list): tf.int64
  )
), because it is not built.


  (_feature_shapes): Dict(
    (city_id_list): TensorShape([64, None, 1])
    (booker_country_list): TensorShape([64, None, 1])
    (hotel_country_list): TensorShape([64, None, 1])
    (weekday_checkin_list): TensorShape([64, None, 1])
    (weekday_checkout_list): TensorShape([64, None, 1])
  )
  (_feature_dtypes): Dict(
    (city_id_list): tf.int64
    (booker_country_list): tf.int64
    (hotel_country_list): tf.int64
    (weekday_checkin_list): tf.int64
    (weekday_checkout_list): tf.int64
  )
), because it is not built.


INFO:tensorflow:Assets written to: /workspace/ensemble/1_predicttensorflowtriton/1/model.savedmodel/assets


INFO:tensorflow:Assets written to: /workspace/ensemble/1_predicttensorflowtriton/1/model.savedmodel/assets
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/utils/tf_utils.py:101: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[key] = tf.keras.utils.serialize_keras_object(maybe_value)
/usr/local/lib/python3.8/dist-packages/merlin/models/tf/core/combinators.py:288: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config[i] = tf.keras.utils.serialize_keras_object(layer)
/usr/local/lib/python3.8/dist-packages/keras/saving/saved_model/layer_serialization.py:134: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return ge

After we export the ensemble, we are ready to start the Triton Inference Server.

The server is installed in Merlin Tensorflow and Merlin PyTorch containers. If you are not using one of our containers, then ensure it is installed in your environment. For more information, see the Triton Inference Server [documentation](https://github.com/triton-inference-server/server/blob/r22.03/README.md#documentation).

You can start the server by running the following command:

```tritonserver --model-repository=/workspace/ensemble/```

For the --model-repository argument, specify the same value as the `export_path` that you specified previously in the `ensemble.export` method.

After you run the `tritonserver` command, wait until your terminal shows messages like the following example:

I0414 18:29:50.741833 4067 grpc_server.cc:4421] Started GRPCInferenceService at 0.0.0.0:8001<br>
I0414 18:29:50.742197 4067 http_server.cc:3113] Started HTTPService at 0.0.0.0:8000<br>
I0414 18:29:50.783470 4067 http_server.cc:178] Started Metrics Service at 0.0.0.0:8002

Let us now package our data for inference. We will send the first 4 rows of our validation data, which corresponds to a single trip. The data will be first processed by the `NVTabular` workflow and subsequentally passed to our transformer model for predicting. 

Let us send the first 4 rows of our validation data to Triton. This will correspond to a single trip (all rows have the same `utrip_id`) with four stops.

In [18]:
from merlin.systems.triton import convert_df_to_triton_input

validation_data = validation_set_dataset.compute()
inputs = convert_df_to_triton_input(wf.input_schema, validation_data.iloc[:4])

In [19]:
import tritonclient.grpc as grpcclient

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer('executor_model', inputs)

The response consists of logits coming from our model.

In [20]:
response.as_numpy('city_id_list/categorical_output')

array([[-4.958611  , -4.977809  ,  0.66012955, ..., -2.32572   ,
        -0.6495375 , -2.4545593 ]], dtype=float32)

In [21]:
predictions = response.as_numpy('city_id_list/categorical_output')
predictions.shape

(1, 37204)

The above values are logits output from the last layer of our model. They correspond in size to the cardinality of `city_id`, our target variable:

In [22]:
cardinality = wf.output_schema['city_id_list'].properties['embedding_sizes']['cardinality']
cardinality

37204

## Summary

We have trained a transformer model for the next item prediction task using language model masking.

For another session-based example that goes deeper into data preprocessing and that covers several advanced techniques (Weight Tying, Temperature Scaling) please see [Session-Based Next Item Prediction for Fashion E-Commerce](https://github.com/NVIDIA-Merlin/models/blob/t4rec_use_case/examples/usecases/ecommerce-session-based-next-item-prediction-for-fashion.ipynb). 